# Similarity Map of WhatsApp Users based

In this tutorial we briefly show how to obtain a similarity map from the users from your WhatsApp group. We will use the library ``whatstk`` included in this project. 

Below we provide some theory background. However, for a detailed and further documentation you can refer to some of these references [Kohonen 1982](http://campus.fi.uba.ar/mod/resource/view.php?id=34864), [Kohonen 1998](http://www.sciencedirect.com/science/article/pii/S0925231298000307), , [Rojas 1996](https://page.mi.fu-berlin.de/rojas/neural/chapter/K15.pdf), [Joschka Boedecker 2015](http://ml.informatik.uni-freiburg.de/_media/documents/teaching/ss15/som.pdf) or simply take a look at the [Wikipedia page](https://en.wikipedia.org/wiki/Self-organizing_map).

## 1. Self-Organizing-Maps
A self-organizing map (SOM) is an unsupervised learning method that performs dimension reduction to a topological-predefined output space.

The picture below (from this [course](http://www.pitt.edu/~is2470pb/Spring05/FinalProjects/Group1a/tutorial/)) illustrates the main idea of SOM

![Image of Yaktocat](http://www.pitt.edu/~is2470pb/Spring05/FinalProjects/Group1a/tutorial/kohonen1.gif)


In this picture, $\boldsymbol{x} = [x_1, \dots, x_n]$ denotes the input vector of features. Note that it only contains one single layer. 

The network is divided into two stages: (1) Competitive Learning and (2) Topologycal output space.

### 1.3 Competitive Learning

Each neuron (unit) $k$ is represented by a _prototype vector_ $\boldsymbol{w}_k$. When feeding the network with the input vector $\boldsymbol{x}$, the closest unit (unit minimizing $||\boldsymbol{x}-\boldsymbol{w}_k||$) is known as the **winning unit**. In a Winner-Takes-All (WTA) approach, only the winning unit prototype vector is updated, i.e.

$$ \Delta \boldsymbol{w}_{win} = \eta(\boldsymbol{x}-\boldsymbol{w}_{win}).$$

Note that in the WTA approach, dead units might easily appear. Thus, at some points we need to allow for dead units to learn a bit in order to start claiming their territory!

### 1.2 Kohonen Map
An alternative to WTA approach relies on allowing other units to be also updated. In particular, all units are updated according to their proximity to the winning units in the output space. Now the update rule is

$$ \Delta \boldsymbol{w}_{k} = \eta h_k(\boldsymbol{x}-\boldsymbol{w}_{k}),$$

where the term $h_k$ quantifies the proximity of the unit $\boldsymbol{w}_k$ to the winning unit in the ouput space (high if they are close) and $\eta$ represent the learning rate.


This simple but powerful idea, allows for easy visualization of high-dimensional data in an eye-friendly format. Typical output spaces are lines, circles or 2D grids (like in the picture).




## 2. Code

Let us now begin this brief tutorial. Let us first import the basic libraries that we will be using.

### 2.1 Initialization

In [1]:
from __future__ import print_function

In [2]:
import sys
sys.path.append('../')

In [3]:
import whatstk.parser as wp

#### Create chat object
We first create a WhatAppChat object of our chat log file. For testing pruposes, we provide a sample chat log file. However, please please feel free to try with your own chats.

In [4]:
chat = wp.WhatsAppChat("../mychats/allan.txt")

#### Obtain some basic data
Let us now obtain basic data from the chat

In [5]:
# Obtain the names of the users from the chat
users = chat.usernames
# Obtain list of days with interventions
days = chat.days
# Obtain number of interventions in the chat
num_interventions = chat.num_interventions

and print it

In [6]:
print("Brief summary")
# Print name of users
print("\n *", len(users),"users found: ")
[print("\t", user) for user in users]
# Number of days the chat has been active
print("\n * Chat was active", len(days), "days")
# Number of interventions
print(" * Chat had", num_interventions, "interventions")
# Average number of interventions per day
int_day=num_interventions/len(days)
print(" * Chat had an average of %.2f" % int_day, "interventions/day")
# Average number of interventions per day per user
int_day_pers = int_day/len(users)
print(" * Chat had an average of %.2f" % int_day_pers, "interventions/day/person")

Brief summary

 * 6 users found: 
	 Allan Jacob
	 Anju
	 Athu
	 Athu Bangalore
	 Chippy
	 Joju Karunya

 * Chat was active 220 days
 * Chat had 4966 interventions
 * Chat had an average of 22.57 interventions/day
 * Chat had an average of 3.76 interventions/day/person


### 2.2 Obtain input data
We start by obtaining the number of interventions of each user per day of chat activity. We can do this by calling the method `interventions_per_day` from the class `WhatsAppChat`, which returns a `DataFrame` of the data (column per username)

In [7]:
# Dataframe containing the number of user interventions per day (only days of chat activity are considered)
interventions_per_day = wp.user_interventions(chat, 'days')

In [8]:
# Show dataframe
interventions_per_day

,Anju,Chippy,Allan Jacob,Joju Karunya,Athu,Athu Bangalore
2015-08-21,1.0,0.0,0.0,0.0,0.0,0.0
2015-08-22,1.0,8.0,5.0,0.0,0.0,0.0
2015-08-23,5.0,14.0,26.0,6.0,0.0,0.0
2015-08-24,6.0,9.0,8.0,0.0,0.0,0.0
2015-08-28,1.0,0.0,1.0,1.0,0.0,0.0
2015-08-30,0.0,0.0,14.0,0.0,0.0,0.0
2015-08-31,10.0,10.0,13.0,6.0,0.0,0.0
2015-09-09,1.0,2.0,0.0,0.0,0.0,0.0
2015-09-17,2.0,0.0,0.0,2.0,0.0,0.0
2015-09-20,2.0,0.0,0.0,1.0,0.0,0.0


To ease the learning, we normalize and center each dimension

In [9]:
# Center each dimension
interventions_per_day = interventions_per_day.sub(interventions_per_day.mean(axis=1), axis=0)
# Normalize each dimension
interventions_per_day = interventions_per_day.divide(interventions_per_day.max(axis=1)-interventions_per_day.min(axis=1), axis=0)
# Show dataframe
interventions_per_day

,Anju,Chippy,Allan Jacob,Joju Karunya,Athu,Athu Bangalore
2015-08-21,0.833333,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667
2015-08-22,-0.166667,0.708333,0.333333,-0.291667,-0.291667,-0.291667
2015-08-23,-0.134615,0.211538,0.673077,-0.096154,-0.326923,-0.326923
2015-08-24,0.240741,0.574074,0.462963,-0.425926,-0.425926,-0.425926
2015-08-28,0.500000,-0.500000,0.500000,0.500000,-0.500000,-0.500000
2015-08-30,-0.166667,-0.166667,0.833333,-0.166667,-0.166667,-0.166667
2015-08-31,0.269231,0.269231,0.500000,-0.038462,-0.500000,-0.500000
2015-09-09,0.250000,0.750000,-0.250000,-0.250000,-0.250000,-0.250000
2015-09-17,0.666667,-0.333333,-0.333333,0.666667,-0.333333,-0.333333
2015-09-20,0.750000,-0.250000,-0.250000,0.250000,-0.250000,-0.250000


### 2.3 Self-Organizing Map

Once we have the our WhatsAppChat object created, we are ready to do have some fun.

In [10]:
from whatstk.learn.som import SelfOrganizingMap

In [11]:
# We define the number of units that we will be using. Large number leads to good global
# fit but poor local fit (low number leads to the oposite)
num_units = 5
# We choose an output space define by an array of neurons arranged in a line fashion
topology = 'line'

In [12]:
# Initialize our SOM
som = SelfOrganizingMap(interventions_per_day, num_units, sigma_initial=num_units/2, num_epochs=1000,
    learning_rate_initial=1, topology=topology)

In [13]:
# Train our SOM
som.train()


* Training *
- Starting parameters: 
	 learning rate = 1
	 sigma = 2.5
- Ending parameters: 
	 learning rate = 0.00034
	 sigma = 0.25058


Finally, we can print the results of the similarity map

In [14]:
som.print_results()


* Results Self Organizing Map *

0 - Anju, Athu
1
2 - Chippy, Joju Karunya
3
4 - Allan Jacob, Athu Bangalore


There are other topologies available. Let us try them.

In [15]:
# Circle topology is the same as line, except that the first and last components are connected
topology = 'circle'
som = SelfOrganizingMap(interventions_per_day, num_units, sigma_initial=num_units/2, num_epochs=1000,
    learning_rate_initial=1, topology=topology)
som.train()
som.print_results()


* Training *
- Starting parameters: 
	 learning rate = 1
	 sigma = 2.5
- Ending parameters: 
	 learning rate = 0.00034
	 sigma = 0.25058

* Results Self Organizing Map *

0
1 - Joju Karunya, Athu Bangalore
2 - Allan Jacob
3 - Chippy
4 - Anju, Athu


In [16]:
# We now try a 2D-grid
# Number of units now denotes the number of units per side. In total, we have 
# num_units*num_units units

num_units = 5 
topology = '2dgrid'
som = SelfOrganizingMap(interventions_per_day, num_units, sigma_initial=num_units/2, num_epochs=1000,
    learning_rate_initial=1, topology=topology)
som.train()
som.print_results()


* Training *
- Starting parameters: 
	 learning rate = 1
	 sigma = 2.5
- Ending parameters: 
	 learning rate = 0.00034
	 sigma = 0.25058

* Results Self Organizing Map *

                0       1  2     3             4
0     Allan Jacob                           Athu
1                  Chippy                       
2                             Anju              
3                                               
4  Athu Bangalore                   Joju Karunya
